In [4]:
import vanna
from vanna.remote import VannaDefault
from vanna.openai import OpenAI_Chat
from vanna.chromadb import ChromaDB_VectorStore
import dotenv
import os
dotenv.load_dotenv()

True

In [5]:
OPENAI_API_KEY = os.getenv('OPENAI_API_KEY')

In [11]:
vn = VannaDefault(model='chinook', api_key=vanna.get_api_key('fduguchunhui@gmail.com'))
vn.connect_to_sqlite('https://vanna.ai/Chinook.sqlite')

In [ ]:
df_ddl = vn.run_sql("SELECT type, sql FROM sqlite_master WHERE sql is not null")

for ddl in df_ddl['sql'].to_list():
  vn.train(ddl=ddl)

In [19]:
class MyVanna(ChromaDB_VectorStore, OpenAI_Chat):
    def __init__(self, config=None):
        ChromaDB_VectorStore.__init__(self, config=config)
        OpenAI_Chat.__init__(self, config=config)

vn = MyVanna(config={'api_key': OPENAI_API_KEY,
                      'model': 'gpt-4o-mini'})

In [17]:
# The following are methods for adding training data. Make sure you modify the examples to match your database.

# DDL statements are powerful because they specify table names, colume names, types, and potentially relationships
vn.train(ddl="""
    CREATE TABLE IF NOT EXISTS my-table (
        id INT PRIMARY KEY,
        name VARCHAR(100),
        age INT
    )
""")

# Sometimes you may want to add documentation about your business terminology or definitions.
vn.train(documentation="Our business defines OTIF score as the percentage of orders that are delivered on time and in full")

# You can also add SQL queries to your training data. This is useful if you have some queries already laying around. You can just copy and paste those from your editor to begin generating new SQL.
vn.train(sql="SELECT * FROM my-table WHERE name = 'John Doe'")

Adding ddl: 
    CREATE TABLE IF NOT EXISTS my-table (
        id INT PRIMARY KEY,
        name VARCHAR(100),
        age INT
    )

Adding documentation....
Question generated with sql: What are the details for the individual named John Doe? 
Adding SQL...


''

In [25]:
# At any time you can inspect what training data the package is able to reference
training_data = vn.get_training_data()
training_data

,id,question,content,training_data_type
0,f2aa8665-80fa-581c-a6d6-723fd746c41f-sql,What are the details for the individual named ...,SELECT * FROM my-table WHERE name = 'John Doe',sql
0,4aafa9de-6443-5409-86f9-5d581327d6a0-ddl,None,\n CREATE TABLE IF NOT EXISTS my-table (\n ...,ddl
